In [2]:
import datetime
import pandas as pd
import os

valid_categories = ["Food", "Travel", "Grocery", "Restaurants", "Medical"]
monthly_budget = {}
monthly_expense_category_sum = {}

def add_expense():
    date = get_date_input()
    category = get_expense_category()
    amount = get_amount_spent()
    description = get_expense_description()

    expense_dict = {
        'Date': date,
        'Category': category,
        'Amount': amount,
        'Description': description
    }
    
    # Print the collected input data
    print("\n--- Expense Added ---")
    print(f"Date: {date}")
    print(f"Category: {category}")
    print(f"Amount: ${amount:.2f}")
    print(f"Description: {description if description else 'N/A'}")

    expenses_list = []
    
    expenses_list.append(expense_dict)    

    expense_df = pd.DataFrame(expenses_list)

    save_expenses(expense_df)
    
    return pd

def view_expense():
    expenses_list = load_expenses()
    for expense in expenses_list:
        print(f"On {expense['Date']}, spent INR {expense['Amount']:.2f} on {expense['Category']} in {expense['Description']}")

def save_expenses(expense_df):
    file_path = 'expenses.csv'
        # Write to file
    if os.path.exists(file_path):
        # File exists, append without writing headers
        expense_df.to_csv(file_path, mode='a', header=False, index=False)
        print(f"Expense added to existing file: {file_path}")
    else:
        # File doesn't exist, create new file with headers
        expense_df.to_csv(file_path, index=False)
        print(f"Created new file: {file_path}")    


        
def load_expenses():
    """Load expenses from file into a list of dictionaries"""
    file_path = 'expenses.csv'
    
    if os.path.exists(file_path):
        # Read the CSV file into a DataFrame
        expenses_df = pd.read_csv(file_path)
        
        expenses_list = expenses_df.to_dict('records')
        print(f"Loaded {len(expenses_list)} expenses from {file_path}")
    else:
        print("No expense file found. Starting with empty list.")
        expenses_list = []
    
    return expenses_list


def set_budget():
    for category in valid_categories:
        monthly_budget.update({
            category: float(input("Enter budget for " + category + ": "))
        })

    print(monthly_budget)

def print_budget():
    print(monthly_budget)

def track_budget():
    expenses_list = load_expenses()
    
    # Create a nested dictionary structure:
    # { year_month: { category: amount, ... }, ... }
    monthly_expenses = {}

    for expense in expenses_list:
        date_parts = expense['Date'].split('-')
        year_month = f"{date_parts[0]}-{date_parts[1]}"
        category = expense['Category']
        amount = expense['Amount']
        
        if year_month not in monthly_expenses:
            monthly_expenses[year_month] = {}
            
        if category not in monthly_expenses[year_month]:
            monthly_expenses[year_month][category] = 0
            
        monthly_expenses[year_month][category] += amount
    
    # Sort by year_month to display in chronological order
    for year_month in sorted(monthly_expenses.keys()):
        
        # Get all categories for this month
        categories = monthly_expenses[year_month]
        
        # Sort categories alphabetically
        for category in sorted(categories.keys()):
            amount = categories[category]
            if(category in monthly_budget and amount>monthly_budget[category]):
                print(f" You have exceeded your budget for category {category} by  INR {(amount-monthly_budget[category]):.2f} for {year_month}")    
            elif(category in monthly_budget):
                print(f" You have INR {(monthly_budget[category]-amount):.2f} left for {year_month} for category {category} ")    
    

def get_expense_description(required=False, min_length=0, max_length=100):
    while True:
        # Prompt user for description
        description = input("Enter expense description: ").strip()
        
        # Check if description is required but empty
        if required and not description:
            print("Description is required. Please enter a description.")
            continue
        
        # Check minimum length (if applicable)
        if description and len(description) < min_length:
            print(f"Description must be at least {min_length} characters long.")
            continue
        
        # Check maximum length
        if len(description) > max_length:
            print(f"Description cannot exceed {max_length} characters. Current length: {len(description)}")
            continue
        
        # If we get here, the description is valid
        return description    

def get_amount_spent(prompt="Enter amount spent: "):
    while True:
        try:
            # Convert to float
            amount = float(input(prompt))
            
            # Validate amount is positive
            if amount <= 0:
                print("Amount must be greater than zero.")
                continue
                
            amount = round(amount, 2)
            
            return amount
            
        except ValueError:
            print("Invalid input. Please enter a valid number.")    



def get_date_input(prompt="Enter a date (YYYY-MM-DD): "):
    while True:
        
        try:
            # Attempt to parse the date
            date_obj = datetime.datetime.strptime(input(prompt), "%Y-%m-%d").date()
            
            # Check if it's a valid date
            datetime.date(date_obj.year, date_obj.month, date_obj.day)
            
            # If we get here, the date is valid
            return date_obj
            
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD format.")

def get_expense_category(prompt="Enter expense category: "):
    
    while True:
        # Get user input and strip whitespace, convert to title case
        category = input(prompt).title()
        
        # Check if the input matches any valid category
        if category in valid_categories:
            return category
        else:
            # Show error message with valid options
            categories_str = ", ".join(valid_categories)
            print(f"Invalid category. Please choose from: {categories_str}")   

def display_menu():
    """Display the main menu options"""
    print("\n===== EXPENSE TRACKER =====")
    print("1. Add Expense")
    print("2. View Expenses")
    print("3. Set Budget")
    print("4. Track Budget")
    print("5. Exit")
    print("==========================")

def main():
    """Main program function with menu-driven interface"""
    # Initialize - load existing data if available
    expenses_list = load_expenses()
    
    while True:
        display_menu()
        
        # Get user choice with error handling
        try:
            choice = int(input("Enter your choice (1-5): "))
        except ValueError:
            print("Invalid input. Please enter a number.")
            continue
            
        # Process the user's choice
        if choice == 1:
            # Add expense
            print("\n--- Add New Expense ---")
            add_expense()
            
        elif choice == 2:
            # View expenses
            print("\n--- View Expenses ---")
            view_expense()
            
        elif choice == 3:
            # Set budget
            print("\n--- Set Monthly Budget ---")
            set_budget()
            
        elif choice == 4:
            # Track budget
            print("\n--- Track Budget ---")
            if not monthly_budget:
                print("No budget set. Please set a budget first.")
                continue
            track_budget()
            
        elif choice == 5:
            # Exit program
            print("\nThank you for using Expense Tracker. Goodbye!")
            break
            
        else:
            # Invalid choice
            print("Invalid choice. Please enter a number between 1 and 5.")
            
        # Pause before showing menu again
        input("\nPress Enter to continue...")


In [3]:
set_budget()

KeyboardInterrupt: Interrupted by user

In [107]:
track_budget()

Loaded 5 expenses from expenses.csv


In [ ]:
main()

Loaded 5 expenses from expenses.csv

===== EXPENSE TRACKER =====
1. Add Expense
2. View Expenses
3. Set Budget
4. Track Budget
5. Exit


Enter your choice (1-5):  1



--- Add New Expense ---


Enter a date (YYYY-MM-DD):  2025-02-01
Enter expense category:  Food
Enter amount spent:  1000
Enter expense description:  Chinese



--- Expense Added ---
Date: 2025-02-01
Category: Food
Amount: $1000.00
Description: Chinese
Expense added to existing file: expenses.csv


...

{'Food': 100000.0, 'Travel': 200000.0, 'Grocery': 300000.0, 'Restaurants': 400000.0, 'Medical': 50000.0}



Press Enter to continue... 



===== EXPENSE TRACKER =====
1. Add Expense
2. View Expenses
3. Set Budget
4. Track Budget
5. Exit


Enter your choice (1-5):  4



--- Track Budget ---
Loaded 6 expenses from expenses.csv
 You have exceeded your budget for category Travel by  INR 300000.00 for 2025-01
 You have INR 99000.00 left for 2025-02 for category Food 
 You have INR 99900.00 left for 2025-05 for category Food 
 You have exceeded your budget for category Travel by  INR 300200.00 for 2025-05
 You have INR 100000.00 left for 2025-06 for category Travel 
